In [3]:
import os
import re
import json
from bs4 import BeautifulSoup

In [4]:
odkud = "downloads/poslanectvo"

In [5]:
kam = "data_raw/poslanectvo"
os.makedirs(kam, exist_ok=True)

In [6]:
stazene = os.listdir(odkud)

In [7]:
def najdi_datum(retezec):
    retezec_cisty = retezec.replace('\xa0', ' ').replace('\u00a0', ' ')
    if 'v roce' not in retezec_cisty:
        datum = re.search(r'\d{1,2}\.\s\d{1,2}\.\s\d{4}', retezec_cisty).group().replace('\xa0',' ')
    else:
        datum = re.search(r'\d{4}', retezec_cisty).group()
    return datum

In [8]:
najdi_datum('Narozena: v roce 1954')

'1954'

In [9]:
def rozlouskni_odrazku(odrazka):
    odrazka_normalizovana = odrazka.text.replace('\xa0', ' ').replace('\u00a0', ' ')
    polozka = {}
    pozice = [' člen',' místopředsed',' předsed']
    for p in pozice:
        if p in odrazka_normalizovana:
            polozka['funkce'] = p.strip()
    if odrazka.find('a'):
        dodatek = ''
        if 'Skupina ČR - ' in odrazka_normalizovana:
            dodatek = 'Skupina ČR - '
        polozka['co'] = f"{dodatek}{odrazka.find('a').text}"
        if ' od ' in odrazka_normalizovana:
            polozka['od'] = najdi_datum(odrazka_normalizovana.split(' od ')[1])
        if ' do ' in odrazka_normalizovana:
            polozka['do'] = najdi_datum(odrazka_normalizovana.split(' do ')[1])
        return polozka

In [10]:
def oscrapuj_poslance(soubor):

    
    with open(os.path.join(odkud, soubor), 'r', encoding='utf-8') as poslanec:
        poslanec = poslanec.read()
        soup = BeautifulSoup(poslanec)
        if 'Volební kraj' in soup.text:
            tabulka = {}
            tabulka['id'] = soubor.split('_')[0]
            tabulka['soubor'] = soubor
            tabulka['jmeno'] = soup.find('title').text
            tabulka['kraj'] = soup.text.split('Volební kraj: ')[1].split(' ')[0]
            if 'poslanec ' in soup.text:
                tabulka['gender'] = 'm'
                tabulka['obdobi_od'] = najdi_datum(soup.text.split('poslanec ')[1][0:100])
                try:
                    tabulka['obdobi_do'] = najdi_datum(soup.text.split('poslanec ')[1].split("Klub")[0][10:100])
                except:
                    tabulka['obdobi_do'] = None
            if 'poslankyně ' in soup.text:
                tabulka['gender'] = 'f'
                tabulka['obdobi_od'] = najdi_datum(soup.text.split('poslankyně ')[1][0:100])
                try:
                    tabulka['obdobi_do'] = najdi_datum(soup.text.split('poslankyně ')[1].split("Klub")[0][10:100])
                except:
                    tabulka['obdobi_do'] = None
            if 'na kandidátce:' in soup.text:
                kandidatka = soup.text.split('na kandidátce:')[1].split('Seznam')[0].strip()
            navrhovatel = None
            if ', navrhovatel' in kandidatka:
                kandidatka, navrhovatel = kandidatka.split(', navrhovatel')[0].strip(), kandidatka.split(', navrhovatel')[1].strip()
            tabulka['kandidatka'], tabulka['navrhovatel'] = kandidatka, navrhovatel
    
            popisek = soup.find(class_='figcaption').text
            if 'naroze' in popisek[0:50].lower():
                tabulka['narozeni'] = najdi_datum(popisek[0:50])
                
            odrazky = soup.find_all('li')
            clenstvi = []
            for o in odrazky:
                try:
                    zpracovana_odrazka = rozlouskni_odrazku(o)
                except:
                    pass
                    # print(f"problém s odrážkou v souboru {soubor}")
                if zpracovana_odrazka != None:
                    if zpracovana_odrazka.get('od'):
                        clenstvi.append(rozlouskni_odrazku(o))
            tabulka['clenstvi'] = clenstvi
        
            return tabulka

In [11]:
oscrapuj_poslance('5903_0.html')

{'id': '5903',
 'soubor': '5903_0.html',
 'jmeno': 'Mgr. Jana Černochová',
 'kraj': 'Hlavní',
 'gender': 'f',
 'obdobi_od': '9. 10. 2021',
 'obdobi_do': None,
 'kandidatka': 'SPOLU',
 'navrhovatel': 'ODS',
 'narozeni': '26. 10. 1973',
 'clenstvi': [{'funkce': 'člen',
   'co': 'Poslanecký klub Občanské demokratické strany',
   'od': '12. 10. 2021'},
  {'funkce': 'člen', 'co': 'Skupina ČR - Izrael', 'od': '2. 3. 2022'},
  {'co': 'Vláda České republiky', 'od': '17. 12. 2021'}]}

In [12]:
stazene = os.listdir(odkud)

for s in stazene:
    try:
        poslanec = oscrapuj_poslance(s)
    except:
        print(s)
        break
    if poslanec != None:
        with open(os.path.join(kam, s.replace('.html','.ndjson')), "w+", encoding="utf-8") as export:
            export.write(json.dumps(poslanec))